In [8]:
!pip install "numpy<2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy
import pandas as pd
# Load built-in MovieLens 100k dataset
data = Dataset.load_builtin('ml-100k')  # UserID, ItemID, Rating

# Define train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
# Use Matrix Factorization (SVD)
model = SVD()

# Train the model
model.fit(trainset)
# Predict on test data
predictions = model.test(testset)

# Calculate metrics
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)
# Get list of all items from the testset for simplicity in this example
all_items = set([item for (_, item, _) in testset])

user_id = '196'  # Example user

# Get inner user ID
inner_user_id = trainset.to_inner_uid(user_id)

# Items user has already rated (inner item IDs)
rated_items_inner = set([item_id for (item_id, _) in trainset.ur[inner_user_id]])

# Get all inner item IDs in the trainset
all_trainset_items_inner = set([item for (_, item, _) in trainset.all_ratings()])

# Items not yet rated by the user (inner item IDs) that are also in the training set
unrated_items_inner = list(all_trainset_items_inner - rated_items_inner)


# Convert inner item IDs back to raw item IDs for prediction if needed,
# but predict takes inner IDs.
# For prediction, we will use the inner item IDs directly.

# Predict ratings for unrated items (using inner item IDs)
predictions = [model.predict(user_id, trainset.to_raw_iid(item_inner_id)) for item_inner_id in unrated_items_inner]


# Sort by highest estimated rating
top_recs = sorted(predictions, key=lambda x: x.est, reverse=True)[:5]

# Show top 5 recommendations
for rec in top_recs:
    print(f"Recommend Item {rec.iid} with predicted rating: {rec.est:.2f}")

RMSE: 0.9392
MAE:  0.7383
Recommend Item 603 with predicted rating: 4.61
Recommend Item 178 with predicted rating: 4.59
Recommend Item 520 with predicted rating: 4.54
Recommend Item 64 with predicted rating: 4.50
Recommend Item 190 with predicted rating: 4.50
